In [1]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm as tq
import warnings
warnings.filterwarnings("ignore")
from glob import glob as glob

In [2]:
class DataPrep():
    def __init__(self, base_folder, update_local_files=False):
        
        # create folder if not exists - used to save prepared data
        if not os.path.exists('Processed Data'):
            os.makedirs('Processed Data')
         
        self.base_folder = base_folder
        self.update_local_files = update_local_files
        
        # list of rich countries fetched from manually created file
        self.rich_countries = pd.read_csv('Rich Countries.csv')['Country Name'].to_list()
        
        # used in preparing column - age
        self.ages_to_keep = ['1-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44',
                             '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '<1', '80-84',
                             '85-89', '90-94', '95+']
        
        # used in preparing column - cause
        self.causes_to_keep = ['HIV/AIDS and sexually transmitted infections',
                               'Interpersonal violence', 
                               'Neoplasms',
                               'Self-harm', 
                               'Chronic respiratory diseases',
                               'Cirrhosis and other chronic liver diseases', 
                               'Transport injuries',
                               'Substance use disorders', 
                               'Diabetes and kidney diseases',
                               'Cardiovascular diseases']
        
        # save file locations for all prepared data
        self.save_file_to = {'mortality': 'Processed Data/mortality.csv',
                              'yll': 'Processed Data/years-of-life-lost.csv',
                              'population': 'Processed Data/population_1990-2019.csv',
                              'standard_population': 'Processed Data/population_usa_2017.csv',
                              'standardized_mortality': 'Processed Data/gbd-rich-countries.csv',
                              'standardized_yll': 'Processed Data/yll-rich-countries.csv',
                              'gdp': 'Processed Data/gdp-pop-pivot.csv'} 
        
        # wrapping functions in decorator - used to save prepared data
        self.prepare_mortality_data = self.dec_SaveFile(self.prepare_mortality_data)
        self.prepare_yll_data = self.dec_SaveFile(self.prepare_yll_data)
        self.prepare_population_data = self.dec_SaveFile(self.prepare_population_data)
        self.prepare_standard_population = self.dec_SaveFile(self.prepare_standard_population)
        self.standardize_mortality_data = self.dec_SaveFile(self.standardize_mortality_data)
        self.standardize_yll_data = self.dec_SaveFile(self.standardize_yll_data)
        self.prepare_gdp_data = self.dec_SaveFile(self.prepare_gdp_data)

    """ decorator """
    def dec_SaveFile(self, func):
        def wrapper(*args, **kwargs):
            flag = False
            if self.update_local_files: 
                flag = True
            result = func(*args, **kwargs, save_file=flag)
            return result
        return wrapper
        
    """ utility function to merge multiple csv files into pandas dataframe, apply row level filters and return dataframe """
    def merge_files(self, file_paths, filters=None, ignore_cols=None):
        dfs = []
        for file in tq(file_paths, desc='Merging Files'):
            df = pd.read_csv(file)
            dfs.append(df.drop(ignore_cols, axis=1))

        dfc = pd.concat(dfs, ignore_index=True)
        if filters:
            for column, values in tq(filters.items(), desc='Applying Filters'):  
                dfc = dfc[dfc[column].apply(lambda x: x in values)]

        return dfc     
    
    """ Prepares Mortality Data - Data from this is later used to apply aggregations to get final data structure """
    def prepare_mortality_data(self, raw_data_path, save_file=None):

        # merge multiple data files into one dataframe
        filters = {'measure': ['Deaths'], 'sex': ['Both'], 'location': self.rich_countries}
        df_deaths = self.merge_files(glob(raw_data_path), filters, ignore_cols=['metric', 'upper', 'lower'])

        # create CN(USA or ORC) column, format years columns (1-4 years => 1-4), keep only required causes and rename all others as "Other"
        df_deaths.rename(columns = {'location':'country', 'val': 'deaths'}, inplace=True)
        df_deaths['CN'] = df_deaths['country'].apply(lambda x: 'USA' if x == 'United States of America' else 'ORC')
        df_deaths['age'] = df_deaths['age'].apply(lambda x: x.split(' ')[0])
        df_deaths['cause'] = df_deaths['cause'].apply(lambda x: 'Other' if x not in self.causes_to_keep else x)

        print(f'\nMortality Data Shape: {df_deaths.shape}\n')
        print(df_deaths.sample(5))

        if save_file: 
            df_deaths.to_csv(self.save_file_to['mortality'], index=False)
            print(f"\nFile Saved at: {self.save_file_to['mortality']}")

        print('- '*40)
        return df_deaths
    
    """ Prepares Years of Life Lost Data - Data from this is later used to apply aggregations to get final data structure """
    def prepare_yll_data(self, raw_data_path, save_file=None):

        # merge multiple data files into one dataframe
        filters = {'measure': ['YLLs (Years of Life Lost)'], 'sex': ['Both'], 'location': self.rich_countries}
        df_yll = self.merge_files(glob(raw_data_path), filters, ignore_cols=['metric', 'upper', 'lower'])

        # create CN(USA or ORC) column, format years columns (1-4 years => 1-4), keep only required causes and rename all others as "Other"
        df_yll.rename(columns = {'location':'country', 'val': 'yll'}, inplace=True)
        df_yll['CN'] = df_yll['country'].apply(lambda x: 'USA' if x == 'United States of America' else 'ORC')
        df_yll['age'] = df_yll['age'].apply(lambda x: x.split(' ')[0])
        df_yll['cause'] = df_yll['cause'].apply(lambda x: 'Other' if x not in self.causes_to_keep else x)

        print(f'\nYears of Life Lost Data Shape: {df_yll.shape}\n')
        print(df_yll.sample(5))

        if save_file: 
            df_yll.to_csv(self.save_file_to['yll'], index=False)
            print(f"\nFile Saved at: {self.save_file_to['yll']}")

        print('- '*40)
        return df_yll
    
    """ Population Data - used in standardizing Mortality and YLL Data """
    def prepare_population_data(self, raw_data_path, save_file=None):

        # merge multiple data files into one dataframe
        filters = {'sex_name': ['both'], 'location_name': self.rich_countries}
        df_pop = self.merge_files(glob(raw_data_path), filters, ignore_cols=['location_id', 'sex_id', 'age_group_id', 'measure_id', 'metric_id'])

        df_pop['age_group_name'] = df_pop['age_group_name'].replace({'1 to 4': '1-4', '5 to 9': '5-9', '10 to 14': '10-14', '15 to 19': '15-19', '20 to 24': '20-24', '25 to 29': '25-29',
                                       '30 to 34': '30-34', '35 to 39': '35-39', '40 to 44': '40-44', '45 to 49': '45-49', '50 to 54': '50-54',
                                       '55 to 59': '55-59', '60 to 64': '60-64', '65 to 69': '65-69', '70 to 74': '70-74', '75 to 79': '75-79',
                                       '<1 year': '<1', '80 to 84': '80-84', '85 to 89': '85-89', '90 to 94': '90-94', '95 plus': '95+'})
        
        # only keep required age
        df_pop = df_pop[df_pop['age_group_name'].isin(self.ages_to_keep)]
        # create CN(USA or ORC) column, remove unnecessary columns
        df_pop['sex_name'] = df_pop['sex_name'].replace({'male': 'Male', 'female': 'Female'})
        df_pop['CN'] = df_pop['location_name'].apply(lambda x: 'USA' if x == 'United States of America' else 'ORC') 
        df_mid = df_pop.drop(columns=['measure_name', 'metric_name', 'upper', 'lower']).copy()
        df = df_mid.rename(columns={'location_name': 'country', 'sex_name': 'sex', 'age_group_name': 'age', 'val': 'pop', 'year_id': 'year'})
        
        print(f'\nPopulation Data Shape: {df.shape}\n')
        print(df.sample(5))

        if save_file: 
            df.to_csv(self.save_file_to['population'], index=False)
            print(f"\nFile Saved at: {self.save_file_to['population']}")

        print('- '*40)
        return df

    """ Standard Population is - USA 2017 """
    def prepare_standard_population(self, country='USA', year=2017, save_file=None):

        # read file created from function "prepare_population_data" and filter by country and year
        df_pop = pd.read_csv('Processed Data/Population_1990-2019.csv')
        df = df_pop[(df_pop['CN'] == country) & (df_pop['year'] == year)]
        
        # apply aggregation
        df = df.groupby(['age', 'year', 'CN']).agg(pop_us17=('pop', 'sum')).reset_index()
        df['weight'] = df['pop_us17']/df['pop_us17'].sum()
        
        print(f'\nStandard Population Data Shape: {df.shape}\n')
        print(df.sample(5))

        if save_file: 
            df.to_csv(self.save_file_to['standard_population'], index=False)
            print(f"\nFile Saved at: {self.save_file_to['standard_population']}")
           
        print('- '*40)
        return df
    
    """ Standardize Mortality data using the standard population """
    def standardize_mortality_data(self, save_file=None):

        # apply aggregations to mortality and population data
        deaths_groupped = self.deaths.groupby(['age', 'year', 'cause', 'CN']).agg({'deaths': 'sum'}).reset_index()
        df_pop_groupped = self.df_pop.groupby(['age', 'year', 'CN']).agg({'pop': 'sum'}).reset_index()

        # join mortality and population data
        df_mid = deaths_groupped.merge(df_pop_groupped, left_on = ['age', 'year', 'CN'], right_on=['age', 'year', 'CN'], how='left')
        df = df_mid.merge(self.pop_us17[['age', 'pop_us17', 'weight']])

        # Deaths Std = standardized(deaths), aa_dr = age adjusted death rate
        df['Deaths Std'] = (df['deaths'] * df['pop_us17'])/df['pop']
        df['Death Rate'] = (df['deaths']/df['pop']) * pow(10,5)
        df['aa_dr'] = ((df['deaths'] * pow(10,5)) / df['pop']) * df['weight']

        print(f'\nStandardized Mortality Data Shape: {df.shape}\n')
        print(df.sample(5))

        if save_file: 
            df.to_csv(self.save_file_to['standardized_mortality'], index=False)
            print(f"\nFile Saved at: {self.save_file_to['standardized_mortality']}")
    
        print('- '*40)
        return df
    
    """ Standardize Years of Life Lost data using the standard population """
    def standardize_yll_data(self, save_file=None):

        # apply aggregations to yll and population data
        yll_groupped = self.yll.groupby(['age', 'year', 'cause', 'CN']).agg({'yll': 'sum'}).reset_index()
        df_pop_groupped = self.df_pop.groupby(['age', 'year', 'CN']).agg({'pop': 'sum'}).reset_index()
        
        # join yll and population data
        df_mid = yll_groupped.merge(df_pop_groupped, left_on=['age', 'year', 'CN'], right_on=['age', 'year', 'CN'], how='left')
        df = df_mid.merge(self.pop_us17[['age', 'pop_us17', 'weight']])

        # aa_yll, aa_yll2 = age adjusted years of life losts
        df['aa_yll'] = ((df['yll'] * pow(10,5)) / df['pop']) * df['weight']
        df['aa_yll2'] = (df['yll'] / (df['pop'] * pow(10,5))) * df['pop_us17']

        print(f'\nStandardized Years of Life Lost Data Shape: {df.shape}\n')
        print(df.sample(5))

        if save_file: 
            df.to_csv(self.save_file_to['standardized_yll'], index=False)
            print(f"\nFile Saved at: {self.save_file_to['standardized_yll']}")
        
        print('- '*40)
        return df
    
    """ GDP data used in visualizing scatter plots """
    def prepare_gdp_data(self, save_file=None):
        
        df_raw = pd.read_excel('P_Data_Extract_From_World_Development_Indicators.xlsx')
        
        # transform data from wide format to long
        old_cols = list(df_raw.columns[4:])
        temp_dct = {}
        for c in old_cols: 
            temp_dct[c] = c[:4]
        df_raw.rename(columns = temp_dct, inplace = 1)
        df_raw.dropna(subset=['Series Code'], inplace=True)
        
        pivot_cols = []
        for k, v in temp_dct.items():
            pivot_cols.append(v)
            
        # split gdp and population data into 2 different dataframes
        df_gdp = df_raw[df_raw['Series Name'] == 'GDP per capita, PPP (current international $)']
        df_pop = df_raw[df_raw['Series Name'] == 'Population, total']
        
        # unpivot population and gdp data
        df_gdp = pd.melt(df_gdp, id_vars = ['Country Name', 'Country Code'], value_vars = pivot_cols, var_name = 'Year', value_name = 'GDP PPP USD')
        df_pop = pd.melt(df_pop, id_vars = ['Country Name', 'Country Code'], value_vars = pivot_cols, var_name = 'Year', value_name = 'Population')
        
        # join the unpivoted dataframes, replace ".." by NaN
        df_processed = df_gdp.merge(df_pop, on = ['Country Name', 'Country Code', 'Year'])
        df_processed['GDP PPP USD'] = df_processed['GDP PPP USD'].replace('..', np.nan)
        df_processed['Population'] = df_processed['Population'].replace('..', np.nan)
        
        # only keep data for years 1990, 2017 and pivot data to get the required format
        df_sc = df_processed[(df_processed['Year'] == '1990') | (df_processed['Year'] == '2017')]
        df_sc = df_sc.pivot(index = 'Country Name', columns = 'Year', values = ['GDP PPP USD', 'Population']).reset_index()
        df_sc.columns = ['_'.join(col) for col in df_sc.columns.values]
        df_sc.rename(columns={'Country Name_': 'Country Name1'}, inplace=True)
        
        print(f'\nPrepared GDP-POP Data Shape: {df_sc.shape}\n')
        print(df_sc.sample(5))

        if save_file: 
            df_sc.to_csv(self.save_file_to['gdp'], index=False)
            print(f"\nFile Saved at: {self.save_file_to['gdp']}")

        print('- '*40)
        return df_sc
    
    """ Prepares Data by running all functions """
    def create_data_extracts(self):
        current_runs = 0
        pbar = tq(total = 7)
        
        pbar.set_description_str(desc='Preparing GDP Data')
        self.gdp = self.prepare_gdp_data()
        pbar.update(current_runs + 1)
        
        pbar.set_description_str(desc='Preparing Mortality Data')
        self.deaths = self.prepare_mortality_data('Raw Data/*/*.csv')
        pbar.update(current_runs + 1)
        
        pbar.set_description_str(desc='Preparing Years of Life Lost Data')
        self.yll = self.prepare_yll_data('Raw Data/*/*.csv')
        pbar.update(current_runs + 1)
        
        pbar.set_description_str(desc='Preparing Population Data')
        self.df_pop = self.prepare_population_data(raw_data_path='Population/*/*.CSV')
        pbar.update(current_runs + 1)
        
        pbar.set_description_str(desc='Preparing Standard Population Data')
        self.pop_us17 = self.prepare_standard_population()
        pbar.update(current_runs + 1)
        
        pbar.set_description_str(desc='Standardizing Mortality Data')
        self.df = self.standardize_mortality_data()
        pbar.update(current_runs + 1)
        
        pbar.set_description_str(desc='Standardizing Years of Life Lost Data')
        self.df = self.standardize_yll_data()
        pbar.update(current_runs + 1)

    """ Loads prepared data from saved files """
    def load_data_extracts(self):
        self.deaths = pd.read_csv(self.save_file_to['mortality'])
        self.yll = pd.read_csv(self.save_file_to['yll'])
        self.df_pop = pd.read_csv(self.save_file_to['population'])
        self.pop_us17 = pd.read_csv(self.save_file_to['standard_population'])
        self.deaths_final = pd.read_csv(self.save_file_to['standardized_mortality'])
        self.yll_final = pd.read_csv(self.save_file_to['standardized_yll'])
        self.gdp = pd.read_csv(self.save_file_to['gdp'])
    
    def run(self):
        if self.update_local_files:
            self.create_data_extracts()
        else:
            self.load_data_extracts()

In [3]:
x = DataPrep(base_folder='My Drive/mac_gdrive/IS', update_local_files=True)
x.run()

y = DataPrep(base_folder='My Drive/mac_gdrive/IS', update_local_files=False)
y.run()

  0%|          | 0/7 [00:00<?, ?it/s]


Prepared GDP-POP Data Shape: (217, 5)

                Country Name1 GDP PPP USD_1990 GDP PPP USD_2017  \
93                    Ireland     13743.542119     78168.967738   
82              Guinea-Bissau      1013.329345      1872.309281   
79                       Guam              NaN              NaN   
72                    Georgia      5727.624172     13589.707392   
200  Turks and Caicos Islands              NaN     25207.831686   

    Population_1990 Population_2017  
93          3513974         4807388  
82           973551         1879826  
79           138263          168606  
72          4802000         3728004  
200           11709           39844  

File Saved at: Processed Data/gdp-pop-pivot.csv
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


Merging Files:   0%|          | 0/35 [00:00<?, ?it/s]

Applying Filters:   0%|          | 0/3 [00:00<?, ?it/s]


Mortality Data Shape: (399840, 8)

         measure  country   sex    age  \
10208272  Deaths    Italy  Both  45-49   
11320852  Deaths  Iceland  Both  75-79   
4699319   Deaths   Norway  Both  50-54   
13916325  Deaths  Austria  Both  50-54   
6184015   Deaths  Iceland  Both    95+   

                                               cause  year     deaths   CN  
10208272                Chronic respiratory diseases  2008  94.094939  ORC  
11320852                                       Other  2012   0.638445  ORC  
4699319                                        Other  1995   0.165632  ORC  
13916325                                       Other  2010   5.333998  ORC  
6184015   Cirrhosis and other chronic liver diseases  1995   0.121326  ORC  

File Saved at: Processed Data/mortality.csv
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 


...

Applying Filters:   0%|          | 0/3 [00:00<?, ?it/s]


Years of Life Lost Data Shape: (399840, 8)

                            measure  country   sex    age  \
17026446  YLLs (Years of Life Lost)  Denmark  Both  75-79   
8454211   YLLs (Years of Life Lost)   France  Both  70-74   
3284599   YLLs (Years of Life Lost)  Iceland  Both  80-84   
971567    YLLs (Years of Life Lost)    Japan  Both  50-54   
297842    YLLs (Years of Life Lost)  Denmark  Both  40-44   

                                               cause  year           yll   CN  
17026446                                   Self-harm  2017    642.689598  ORC  
8454211   Cirrhosis and other chronic liver diseases  1990  20962.809062  ORC  
3284599                                        Other  2005      0.284896  ORC  
971567                  Diabetes and kidney diseases  1994  32736.097277  ORC  
297842                  Diabetes and kidney diseases  2006   1096.259719  ORC  

File Saved at: Processed Data/years-of-life-lost.csv
- - - - - - - - - - - - - - - - - - - - - - - - - - - 

Merging Files:   0%|          | 0/70 [00:00<?, ?it/s]

Applying Filters:   0%|          | 0/2 [00:00<?, ?it/s]


Population Data Shape: (41160, 6)

           country   sex    age  year            pop   CN
4481067  Australia  both  50-54  1973  718070.552866  ORC
6576276    Iceland  both    5-9  1994   20600.339706  ORC
5814313    Denmark  both  40-44  1999  372768.720607  ORC
5626610     Canada  both    95+  1963    3625.849181  ORC
960974     Iceland  both  85-89  1988    1841.449339  ORC

File Saved at: Processed Data/population_1990-2019.csv
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Standard Population Data Shape: (21, 5)

      age  year   CN      pop_us17    weight
8   45-49  2017  USA  2.110497e+07  0.065028
2   15-19  2017  USA  2.158521e+07  0.066508
20     <1  2017  USA  3.801319e+06  0.011713
5   30-34  2017  USA  2.171028e+07  0.066893
4   25-29  2017  USA  2.280305e+07  0.070260

File Saved at: Processed Data/population_usa_2017.csv
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

Standardized Mortality Data Sh